In [1]:
from os import listdir
from os.path import join
import scipy.io as sio
import numpy as np
import networkx as nx
# from scipy.linalg import block_diag

In [2]:
files_path = r"C:\F\Work\Visualization\ssget\mat"
files = [join(files_path,f) for f in listdir(files_path)]

In [3]:
i = 0
num_vertex = []
num_edge = []
for file in files:
    try:
        matrix = sio.loadmat(file)['Problem'][0][0][1]
    except:
        continue
    if type(matrix).__module__ != 'scipy.sparse.csc':
        matrix = sio.loadmat(file)['Problem'][0][0][2]
    try:
        matrix = np.where(matrix.toarray()==0,0,1)
    except:
        continue
    if not np.array_equal(matrix,matrix.T):
        continue
    graph = nx.Graph(matrix)
    for l in range(matrix.shape[0]):
        matrix[l,l] = 0
    if nx.is_connected(graph):
        np.save(r"C:\F\Work\Visualization\real_data\matrix"+str(i) ,matrix)
        i += 1
        num_vertex.append(matrix.shape[0])
        num_edge.append(np.sum(matrix)/2)

In [8]:
para = np.random.multivariate_normal([np.mean(num_vertex),np.mean(num_edge)],
                                             np.cov([num_vertex,num_edge]),size=10000)
idx0 = para[:,0] > min(num_vertex)
idx1 = para[:,1] > min(num_edge)
idx2 = para[:,0] < max(num_vertex)
idx3 = para[:,1] < max(num_edge)
idx4 = (para[:,1]-(para[:,0]*(para[:,0]-1)/2)) < 0
idx = [idx0[i] and idx1[i] and idx2[i] and idx3[i] and idx4[i] for i in range(len(para))]
para = np.round(para[idx])

In [18]:
from layout import *

In [14]:
from knn import * 
from graphlet import *

features = []
m = []
for n,i in enumerate(para):
    matrix = create_matrix(int(i[0]), int(i[1]))
    features.append(RW(matrix))
    m.append(matrix)
    if n%100 == 0:
        print(n)

features = np.array(features)
files_path = r"C:\F\Work\Visualization\real_data"
files = [join(files_path,f) for f in listdir(files_path)]

for i,f in enumerate(files):
    test = np.load(f)
    test_feature = RW(test)
    idx = knn(features, test_feature)
    draw(m[idx], 'estimated_layout_'+str(i), True, None)
    draw(test, 'real_layout_'+str(i), True, None)